In [1]:
from rtlsdr import RtlSdr,limit_time
import numpy as np
import matplotlib.pyplot as plt
import time
from threading import Thread
import cantools
import can
import random

In [2]:
sdr = RtlSdr() 
sdr.sample_rate = 2.4e6 # Hz
sdr.center_freq = 15e6   # Hz
sdr.set_direct_sampling('q')
sdr.set_agc_mode(True)

Found Rafael Micro R820T tuner
[R82XX] PLL not locked!
Enabled direct sampling mode, input 2


0

In [3]:
# Parse CAN Database file
db = cantools.database.load_file('dbc/busmaster_ramn.dbc')
print(db.messages)

[message('Brake', 0x24, False, 8, None), message('Accelerator', 0x39, False, 8, None), message('Steering', 0x62, False, 8, None), message('Shift', 0x77, False, 8, None), message('Horn', 0x98, False, 8, None), message('LightSwitch', 0x150, False, 8, None), message('TurnIndicators', 0x1a7, False, 8, None), message('EngineKey', 0x1b8, False, 8, None), message('LEDStatus', 0x1bb, False, 8, None), message('ParkingBrake', 0x1d3, False, 8, None)]


In [4]:
# Generate a list of messages randomly selected from the database
num_messages = 1000
messages = []
for i in range(num_messages):
    # Choose a random message from the DBC file
    message = random.choice(db.messages)
    # Generate random data for the message
    data = {}
    for signal in message.signals:
        min_value = signal.minimum or 0
        max_value = signal.maximum or 2**signal.length - 1
        if signal.is_signed:
            min_value = max(min_value, -2**(signal.length - 1))
            max_value = min(max_value, 2**(signal.length - 1) - 1)
        data[signal.name] = random.randint(round(min_value), round(max_value))

    # Encode the message
    messages.append((message.frame_id,message.encode(data, strict=False)))

print(f"Generated {num_messages} random CAN messages from the dbc file")

Generated 1000 random CAN messages from the dbc file


In [5]:
for message in messages[:10]:
    print(message)

(98, b'\x00R\x96\x8d\xc3{y)')
(98, b'\x08zN\xc8\xf2\xf9\xa6\x88')
(57, b'\x01q\x0f\xcb\x98\xc3\xe4\x17')
(119, b'\xc4b\xabH\xf51\x01s')
(443, b"\x00\x00U\x19\xc8'\xf3\r")
(467, b'\x00\x00\x85\xfe\xe3Ej\xde')
(36, b'\n\xe1G\x07\x83\xa4\xa7\x8f')
(467, b'\x00\x00\x93W\xc6\xa1\x0c$')
(152, b'\x00\x00I\x80\xaba\x9a;')
(336, b'\x06\x00\xfcBv\xfb\x1a>')


In [6]:
def write_to_file_thread(samples,starttime,endtime,filename):
    n_samples = len(samples)
    with open(filename,'a') as file:
        db_values = 20 * np.log10(np.abs(samples))
        time_axis = np.linspace(starttime,endtime, n_samples + 1)[1:]
        for i in range(n_samples):
            file.write(f"{time_axis[i]};{db_values[i]}\n")

def write_samples_to_file(samples, sdr):
    if sdr.count == 0:
        sdr.count += 1
        sdr.can_bus.send(sdr.packet)
    elif sdr.count > 15:
        sdr.cancel_read_async()
    else:
        sdr.count += 1
        endtime = time.time()
        file_write_thread = Thread(target=write_to_file_thread, args=[samples, sdr.starttime, endtime, sdr.filename])
        file_write_thread.start()
        sdr.starttime = endtime

# Send the randomly generated CAN messages
can_bus = can.interface.Bus('can0',interface="socketcan")
for message in messages:
    sdr.filename = f"{time.time()}-sdr-samples.csv"
    sdr.starttime = time.time()
    sdr.count = 0
    sdr.can_bus = can_bus
    with open(sdr.filename,'w') as file:
        file.write(f"CAN Message; 0x{message[0]:02x}:{message[1].hex()}\n")
        file.write("Timestamp;Amplitude\n")
    packet = can.Message(arbitration_id=message[0], data=message[1])
    sdr.packet = packet
    sdr.read_samples_async(write_samples_to_file,1024,sdr)
    # can_bus.send(packet)
    time.sleep(0.2)
can_bus.shutdown()


Allocating 15 zero-copy buffers


Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocating 15 zero-copy buffers
Allocati

: 